# Packages and Imports

1. Outline Snowpark Architecture:

    1. Types of Libraries(DataFrames, MachineLearning)

            Anaconnda repository and Other-third party libraries

2. Operationalize UDF's and UDTF's in Snowpark

    Use Python Modules(packaged python code)with UDF's

3. Operationalize Snowpark stored Procedures

    Use Python Modules with Stored Procedures

    Make Dependencies available to code

    Bring Python Modules to be used with UDF's

        Stored Procedures to enable resuse of code

For more infomation follow the below links:

1. [Using third-party packages](https://docs.snowflake.com/en/developer-guide/udf/python/udf-python-packages)

2. [How to use third party Python packages not published on Snowflake's Anaconda channel in Snowpark.](https://community.snowflake.com/s/article/how-to-use-other-python-packages-in-snowpark)

3. [How to import custom Python module into Snowflake and use it in Snowpark UDF?](https://stackoverflow.com/questions/76946415/how-to-import-custom-python-module-into-snowflake-and-use-it-in-snowpark-udf)



In [ ]:
from snowflake.snowpark.functions import sproc
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session

session = get_active_session()
session.custom_package_usage_config = {
    "enabled": True,
    "cache_path": "@stage1"
}

In [ ]:
# registered SP, passing the module locally (still TODO! try to fix it!)
# must upload secrets.py file in this notebook!
session.add_import("secrets.py")

@sproc(name="showSecrets",
    replace=True,
    is_permanent=True,
    packages=["snowflake-snowpark-python"],
    stage_location="@stage1")
def showSecrets2(session: Session) -> str:
    import secrets
    return secrets.show()

session.call("showSecrets")

In [ ]:
# registered SP, passing the module through the stage (still TODO! try to fix it!)
# must upload secrets.py file in this notebook!
session.file.put("secrets.py", "@stage1", auto_compress=False)

@sproc(name="showSecrets2",
    replace=True,
    is_permanent=True,
    packages=["snowflake-snowpark-python"],
    imports=["@stage1/secrets.py"],
    stage_location="@stage1")
def showSecrets2(session: Session) -> str:
    import sys
    import_dir = sys._xoptions.get("snowflake_import_directory")
    sys.path.append(f"{import_dir}secrets.py")
    import secrets
    return secrets.show()

session.call("showSecrets2")